In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 134.9 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
# common imports

import sys
assert sys.version_info >= (3, 5)


import sklearn
assert sklearn.__version__ >= "0.20"


import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os

np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

2024-05-09 19:10:39.361729: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 19:10:39.361768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 19:10:39.363050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 19:10:39.369351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 19:10:39.914032: W tensorflow/compiler/tf2

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
import datasets

dataset = datasets.load_from_disk('../../datasets/ARID_supporting_scripts/5_1_training_set')

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['REQID', 'REQID_expanded', 'Requirement Sentences', 'Open/ Closed Source', 'class', 'signal_keyword', 'Source', 'label'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['REQID', 'REQID_expanded', 'Requirement Sentences', 'Open/ Closed Source', 'class', 'signal_keyword', 'Source', 'label'],
        num_rows: 480
    })
})

In [10]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


batch_size = 16


lbl_ = dataset['train'].features['label'].names
label2id = {lbl: idx for idx, lbl in enumerate(lbl_)}
id2label = {val: key for key, val in label2id.items()}
id2label

model_ckpt = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = TFAutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                        num_labels = len(lbl_),
                                                        id2label = id2label,
                                                        label2id = label2id,)

/home/kasra/metal-engine/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-05-09 19:10:55.542756: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 19:10:55.543030: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-09 19:10:55.543981: I ex

In [12]:
def preprocess_function(dataset):
    return tokenizer(dataset['Requirement Sentences'], truncation = True)

In [13]:
X_train_encoded = dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/1916 [00:00<?, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

In [14]:
print(X_train_encoded['train']['Requirement Sentences'][0])
print(X_train_encoded['train']['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(X_train_encoded['train']['input_ids'][0]))

The DWA must request DWA acknowledgment flashing when the DWA has assumed the "armed" state and the outer skin is closed.
[0, 133, 211, 8460, 531, 2069, 211, 8460, 38169, 22643, 77, 5, 211, 8460, 34, 9159, 5, 22, 17651, 113, 194, 8, 5, 15705, 3024, 16, 1367, 4, 2]
['<s>', 'The', 'ĠD', 'WA', 'Ġmust', 'Ġrequest', 'ĠD', 'WA', 'Ġacknowledgment', 'Ġflashing', 'Ġwhen', 'Ġthe', 'ĠD', 'WA', 'Ġhas', 'Ġassumed', 'Ġthe', 'Ġ"', 'armed', '"', 'Ġstate', 'Ġand', 'Ġthe', 'Ġouter', 'Ġskin', 'Ġis', 'Ġclosed', '.', '</s>']


In [15]:
X_train_encoded

DatasetDict({
    train: Dataset({
        features: ['REQID', 'REQID_expanded', 'Requirement Sentences', 'Open/ Closed Source', 'class', 'signal_keyword', 'Source', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1916
    })
    test: Dataset({
        features: ['REQID', 'REQID_expanded', 'Requirement Sentences', 'Open/ Closed Source', 'class', 'signal_keyword', 'Source', 'label', 'input_ids', 'attention_mask'],
        num_rows: 480
    })
})

In [16]:
tf_train_dataset = model.prepare_tf_dataset(
    X_train_encoded['train'],
    shuffle = True,
    batch_size = batch_size,
    tokenizer = tokenizer
)

tf_valid_dataset = model.prepare_tf_dataset(
    X_train_encoded['test'],
    shuffle = False,
    batch_size = batch_size,
    tokenizer = tokenizer
)

In [17]:
from transformers import create_optimizer

num_epochs = 30
batches_per_epoch = len(X_train_encoded['train']) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr = 2e-5, num_warmup_steps = 0, num_train_steps = total_train_steps
)

In [18]:
import evaluate
from transformers.keras_callbacks import KerasMetricCallback


def compute_metrics(eval_predictions):
    metric1 = evaluate.load("precision")
    metric2 = evaluate.load("recall")
    metric3 = evaluate.load("f1")


    predictions, labels = eval_predictions
    predictions = np.argmax(predictions, axis = 1)

    precision = metric1.compute(predictions = predictions, references = labels, average = 'macro')["precision"]
    recall = metric2.compute(predictions = predictions, references = labels, average = 'macro')["recall"]
    f1 = metric3.compute(predictions = predictions, references = labels, average = 'macro')["f1"]
    return {"precision": precision, "recall": recall, "f1": f1}

metric_callback = KerasMetricCallback(metric_fn = compute_metrics, eval_dataset = tf_valid_dataset)

In [19]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

model_name = model_ckpt.split("/")[-1]
print(model_name)
push_to_hub_model_id = f'{model_name}-finetuned-iso29148-req-detector'
tensorboard_callback = TensorBoard(log_dir="./requirement_detector_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir = "./requirement_detector_model_save",
    tokenizer = tokenizer,
    hub_model_id = push_to_hub_model_id,
)

callbacks = [push_to_hub_callback, tensorboard_callback, metric_callback]

/home/kasra/metal-engine/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/home/kasra/classification/requirement_detector_model_save is already a clone of https://huggingface.co/kasrahabib/roberta-base-finetuned-iso29148-req-detector. Make sure you pull the latest changes with `repo.git_pull()`.


roberta-base


In [20]:
model.compile(optimizer = optimizer)
history = model.fit(tf_train_dataset, validation_data = (tf_valid_dataset), epochs = num_epochs, callbacks = callbacks)

Epoch 1/30


2024-05-09 19:11:20.486812: I external/local_xla/xla/service/service.cc:168] XLA service 0x7bd188952750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-09 19:11:20.486853: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-05-09 19:11:20.486863: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-05-09 19:11:20.494859: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-09 19:11:20.522049: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1715274680.598352  810694 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


119/119 [==============================] - 62s 312ms/step - loss: 2.4054 - val_loss: 1.3529 - precision: 0.4290 - recall: 0.4778 - f1: 0.4136
Epoch 2/30


/home/kasra/metal-engine/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


119/119 [==============================] - ETA: 0s - loss: 1.0479

...

119/119 [==============================] - 43s 366ms/step - loss: 0.0251 - val_loss: 0.4190 - precision: 0.9107 - recall: 0.9015 - f1: 0.9046
Epoch 30/30
119/119 [==============================] - ETA: 0s - loss: 0.0257

Several commits (30) will be pushed upstream.


119/119 [==============================] - 44s 370ms/step - loss: 0.0257 - val_loss: 0.4146 - precision: 0.9137 - recall: 0.9086 - f1: 0.9100


Several commits (31) will be pushed upstream.
The progress bars may be unreliable.
EOF
error: failed to push some refs to 'https://huggingface.co/kasrahabib/roberta-base-finetuned-iso29148-req-detector'



OSError: EOF
error: failed to push some refs to 'https://huggingface.co/kasrahabib/roberta-base-finetuned-iso29148-req-detector'


In [21]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_folder(
    folder_path = "./requirement_detector_model_save",
    repo_id = "kasrahabib/" + push_to_hub_model_id,
    repo_type = "model",
)

events.out.tfevents.1715274594.iste.810010.0.v2:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

events.out.tfevents.1715274665.iste.810516.0.v2:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1715274710.iste.810516.1.v2:   0%|          | 0.00/4.75k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kasrahabib/roberta-base-finetuned-iso29148-req-detector/commit/95a8d774cfadb889cd64fdf1f0d017f27d7eaf0f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='95a8d774cfadb889cd64fdf1f0d017f27d7eaf0f', pr_url=None, pr_revision=None, pr_num=None)